In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from skimage.io import imread
from skimage.transform import resize
import os

**Preprocessing Data**

In [ ]:
import warnings
warnings.filterwarnings('ignore')
datadir = '/content/drive/MyDrive/ML/Projects/Dataset1'
categories = ['cake','pizza']

target = []
flat_data = []

for i in categories:
  path = os.path.join(datadir,i)
  for img in os.listdir(path):
    img_array = imread(os.path.join(path,img))
    img_resize = resize(img_array,(150,150,3))
    flat_data.append(np.ndarray.flatten(img_resize))
    target.append(i)
flat_data = np.array(flat_data)
target = np.array(target)

**Creating Dataframe**

In [ ]:
df = pd.DataFrame(flat_data)
df['Target'] = target

#Display head of Dataframe
df.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,...,67461,67462,67463,67464,67465,67466,67467,67468,67469,67470,67471,67472,67473,67474,67475,67476,67477,67478,67479,67480,67481,67482,67483,67484,67485,67486,67487,67488,67489,67490,67491,67492,67493,67494,67495,67496,67497,67498,67499,Target
0,0.158082,0.100575,0.068570,0.200382,0.147348,0.104931,0.217438,0.170379,0.115558,0.360842,0.314156,0.251411,0.356314,0.321020,0.261402,0.325648,0.288079,0.235111,0.331974,0.281690,0.222431,0.319757,0.270954,0.204531,0.326038,0.282900,0.206771,0.345867,0.302730,0.227660,0.359163,0.323869,0.254771,0.368441,0.333147,0.274275,0.384654,0.363490,0.331712,0.473132,...,0.644288,0.642732,0.625425,0.638170,0.639167,0.596205,0.624889,0.635163,0.551692,0.626133,0.642525,0.607507,0.730121,0.757361,0.819585,0.777822,0.834510,0.939148,0.802824,0.873298,0.975808,0.820104,0.890692,0.983005,0.806379,0.880889,0.978850,0.804915,0.882536,0.981386,0.793754,0.861150,0.962719,0.776183,0.836340,0.927869,0.762044,0.811529,0.897804,cake
1,0.866667,0.807843,0.686917,0.867038,0.808215,0.687457,0.871529,0.812706,0.695059,0.880239,0.821416,0.706880,0.869778,0.807843,0.689935,0.865856,0.799778,0.680692,0.869359,0.802693,0.691660,0.873699,0.798797,0.697229,0.886275,0.807843,0.711817,0.871163,0.800288,0.709804,0.861502,0.790914,0.701438,0.869010,0.799033,0.704304,0.878296,0.811630,0.709150,0.873699,...,0.050980,0.019608,0.070723,0.050170,0.020375,0.060967,0.054902,0.027451,0.058013,0.050693,0.023758,0.047059,0.054902,0.031373,0.047453,0.058512,0.024421,0.043218,0.071260,0.025733,0.041419,0.094275,0.041255,0.056941,0.136959,0.058676,0.070327,0.146850,0.062745,0.064497,0.144288,0.061935,0.058013,0.145845,0.061255,0.064674,0.141176,0.054902,0.066667,cake
2,0.465046,0.379582,0.258656,0.444627,0.376163,0.253720,0.438405,0.379582,0.261962,0.440605,0.379944,0.266218,0.443573,0.379582,0.273264,0.438144,0.375399,0.269516,0.431373,0.369856,0.253994,0.427451,0.368627,0.239634,0.425542,0.368627,0.228261,0.446484,0.380105,0.247294,0.448589,0.374079,0.249400,0.438605,0.364095,0.239416,0.435159,0.360784,0.235294,0.429568,...,0.913725,0.949020,0.929412,0.913725,0.949020,0.929412,0.914536,0.949830,0.930222,0.917647,0.952941,0.933333,0.919556,0.954850,0.935242,0.921569,0.956863,0.937255,0.921569,0.956863,0.937255,0.925490,0.960784,0.941176,0.924680,0.959974,0.940366,0.925490,0.960784,0.941176,0.930195,0.969411,0.938169,0.933333,0.972549,0.941176,0.933333,0.972549,0.937255,cake
3,0.734144,0.714536,0.636105,0.741711,0.722104,0.643672,0.738118,0.718510,0.640078,0.743405,0.723798,0.644504,0.745098,0.728984,0.643565,0.738954,0.722196,0.640288,0.745961,0.714954,0.651573,0.745098,0.712863,0.651843,0.745098,0.717647,0.645908,0.752654,0.717647,0.639216,0.752764,0.712685,0.644471,0.745524,0.706745,0.658069,0.733477,0.702614,0.650980,0.733333,...,0.767974,0.749020,0.701961,0.785931,0.755895,0.715343,0.790824,0.749020,0.726176,0.795791,0.756288,0.736967,0.789805,0.760341,0.729412,0.788235,0.757725,0.734196,0.784314,0.752941,0.738118,0.780392,0.749020,0.741803,0.772000,0.745098,0.749454,0.758137,0.738529,0.723287,0.749122,0.733333,0.695082,0.733740,0.716563,0.694118,0.710614,0.685521,0.688632,cake
4,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.007843,0.023085,0.006745,0.000654,0.100362,...,0.507399,0.577333,0.561791,0.496422,0.562984,0.548915,0.505412,0.546138,0.539843,0.499253,0.527567,0.520235,0.477961,0.498956,0.484758,0.482659,0.476157,0.472706,0.487529,0.478449,0.481778,0.474275,0.488680,0.485831,0.492899,0.513369,0.506075,0.498039,0.546379,0.535981,0.510405,0.554769,0.553673,0.527922,0.552941,0.567137,0.522379,0.564580,0.576523,cake


**Label Encoding**

In [ ]:
# Encoding cake as 0 and pizza as 1
from sklearn.preprocessing import LabelEncoder
labelencoder = LabelEncoder()
df['Target_Label'] = labelencoder.fit_transform(df['Target'])
X = df.iloc[:,:-2].values
y = df.iloc[:,-1].values
df.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,...,67462,67463,67464,67465,67466,67467,67468,67469,67470,67471,67472,67473,67474,67475,67476,67477,67478,67479,67480,67481,67482,67483,67484,67485,67486,67487,67488,67489,67490,67491,67492,67493,67494,67495,67496,67497,67498,67499,Target,Target_Label
0,0.158082,0.100575,0.068570,0.200382,0.147348,0.104931,0.217438,0.170379,0.115558,0.360842,0.314156,0.251411,0.356314,0.321020,0.261402,0.325648,0.288079,0.235111,0.331974,0.281690,0.222431,0.319757,0.270954,0.204531,0.326038,0.282900,0.206771,0.345867,0.302730,0.227660,0.359163,0.323869,0.254771,0.368441,0.333147,0.274275,0.384654,0.363490,0.331712,0.473132,...,0.642732,0.625425,0.638170,0.639167,0.596205,0.624889,0.635163,0.551692,0.626133,0.642525,0.607507,0.730121,0.757361,0.819585,0.777822,0.834510,0.939148,0.802824,0.873298,0.975808,0.820104,0.890692,0.983005,0.806379,0.880889,0.978850,0.804915,0.882536,0.981386,0.793754,0.861150,0.962719,0.776183,0.836340,0.927869,0.762044,0.811529,0.897804,cake,0
1,0.866667,0.807843,0.686917,0.867038,0.808215,0.687457,0.871529,0.812706,0.695059,0.880239,0.821416,0.706880,0.869778,0.807843,0.689935,0.865856,0.799778,0.680692,0.869359,0.802693,0.691660,0.873699,0.798797,0.697229,0.886275,0.807843,0.711817,0.871163,0.800288,0.709804,0.861502,0.790914,0.701438,0.869010,0.799033,0.704304,0.878296,0.811630,0.709150,0.873699,...,0.019608,0.070723,0.050170,0.020375,0.060967,0.054902,0.027451,0.058013,0.050693,0.023758,0.047059,0.054902,0.031373,0.047453,0.058512,0.024421,0.043218,0.071260,0.025733,0.041419,0.094275,0.041255,0.056941,0.136959,0.058676,0.070327,0.146850,0.062745,0.064497,0.144288,0.061935,0.058013,0.145845,0.061255,0.064674,0.141176,0.054902,0.066667,cake,0
2,0.465046,0.379582,0.258656,0.444627,0.376163,0.253720,0.438405,0.379582,0.261962,0.440605,0.379944,0.266218,0.443573,0.379582,0.273264,0.438144,0.375399,0.269516,0.431373,0.369856,0.253994,0.427451,0.368627,0.239634,0.425542,0.368627,0.228261,0.446484,0.380105,0.247294,0.448589,0.374079,0.249400,0.438605,0.364095,0.239416,0.435159,0.360784,0.235294,0.429568,...,0.949020,0.929412,0.913725,0.949020,0.929412,0.914536,0.949830,0.930222,0.917647,0.952941,0.933333,0.919556,0.954850,0.935242,0.921569,0.956863,0.937255,0.921569,0.956863,0.937255,0.925490,0.960784,0.941176,0.924680,0.959974,0.940366,0.925490,0.960784,0.941176,0.930195,0.969411,0.938169,0.933333,0.972549,0.941176,0.933333,0.972549,0.937255,cake,0
3,0.734144,0.714536,0.636105,0.741711,0.722104,0.643672,0.738118,0.718510,0.640078,0.743405,0.723798,0.644504,0.745098,0.728984,0.643565,0.738954,0.722196,0.640288,0.745961,0.714954,0.651573,0.745098,0.712863,0.651843,0.745098,0.717647,0.645908,0.752654,0.717647,0.639216,0.752764,0.712685,0.644471,0.745524,0.706745,0.658069,0.733477,0.702614,0.650980,0.733333,...,0.749020,0.701961,0.785931,0.755895,0.715343,0.790824,0.749020,0.726176,0.795791,0.756288,0.736967,0.789805,0.760341,0.729412,0.788235,0.757725,0.734196,0.784314,0.752941,0.738118,0.780392,0.749020,0.741803,0.772000,0.745098,0.749454,0.758137,0.738529,0.723287,0.749122,0.733333,0.695082,0.733740,0.716563,0.694118,0.710614,0.685521,0.688632,cake,0
4,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.007843,0.023085,0.006745,0.000654,0.100362,...,0.577333,0.561791,0.496422,0.562984,0.548915,0.505412,0.546138,0.539843,0.499253,0.527567,0.520235,0.477961,0.498956,0.484758,0.482659,0.476157,0.472706,0.487529,0.478449,0.481778,0.474275,0.488680,0.485831,0.492899,0.513369,0.506075,0.498039,0.546379,0.535981,0.510405,0.554769,0.553673,0.527922,0.552941,0.567137,0.522379,0.564580,0.576523,cake,0


**Splitting the Data**

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.25, random_state=1) # 0.25 x 0.8 = 0.2

**Normalizing data**

In [ ]:
X_train /= 255
X_test /= 255

**Creating a Neural Network**

*   256 neurons in input layer
*   128 neurons in 1st hidden layer
*   64 neurons in 2nd hidden layer
*   32 neurons in 3rd hidden layer
*   1 neuron in output layer

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
ann_model = Sequential()

# Input Layer
ann_model.add(Dense(256,input_dim = len(X_train[0,:]),activation='relu'))  

# Hidden Layer
ann_model.add(Dense(128,activation='relu'))  

# Hidden Layer
ann_model.add(Dense(64,activation='relu')) 

# Hidden Layer
ann_model.add(Dense(32,activation='relu'))

# Output Layer
ann_model.add(Dense(1,activation='sigmoid'))
print(ann_model.summary())

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 256)               17280256  
_________________________________________________________________
dense_1 (Dense)              (None, 128)               32896     
_________________________________________________________________
dense_2 (Dense)              (None, 64)                8256      
_________________________________________________________________
dense_3 (Dense)              (None, 32)                2080      
_________________________________________________________________
dense_4 (Dense)              (None, 1)                 33        
Total params: 17,323,521
Trainable params: 17,323,521
Non-trainable params: 0
_________________________________________________________________
None


**Compiling the Model**

In [ ]:
ann_model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])

**Training the Model**

In [ ]:
history = ann_model.fit(X_train, y_train, validation_data=(X_val,y_val),batch_size = 200, epochs = 100)

Epoch 1/100
2/2 [==============================] - 2s 742ms/step - loss: 0.6928 - accuracy: 0.4994 - val_loss: 1.1911 - val_accuracy: 0.7167
Epoch 2/100
2/2 [==============================] - 1s 280ms/step - loss: 0.6842 - accuracy: 0.7594 - val_loss: 4.6437 - val_accuracy: 0.7083
Epoch 3/100
2/2 [==============================] - 0s 273ms/step - loss: 0.6616 - accuracy: 0.7750 - val_loss: 5.5547 - val_accuracy: 0.8250
Epoch 4/100
2/2 [==============================] - 0s 274ms/step - loss: 0.6355 - accuracy: 0.8004 - val_loss: 11.1046 - val_accuracy: 0.7500
Epoch 5/100
2/2 [==============================] - 0s 274ms/step - loss: 0.5994 - accuracy: 0.8074 - val_loss: 15.3075 - val_accuracy: 0.8000
Epoch 6/100
2/2 [==============================] - 0s 279ms/step - loss: 0.5586 - accuracy: 0.8196 - val_loss: 21.8391 - val_accuracy: 0.8167
Epoch 7/100
2/2 [==============================] - 0s 256ms/step - loss: 0.5104 - accuracy: 0.8228 - val_loss: 30.0112 - val_accuracy: 0.8250
Epoch 8/1

In [ ]:
# Save Model
ann_model.save('/content/drive/MyDrive/ML/Projects/saved_model/annModel.h5')

**Evaluating the Model**

In [ ]:
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
prediction = ann_model.predict(X_val)
accuracy_score(y_val,prediction.round())*100

80.0

In [ ]:
print(confusion_matrix(y_val,prediction.round()))

[[55 15]
 [ 9 41]]


In [ ]:
print(classification_report(y_val,prediction.round()))

              precision    recall  f1-score   support

           0       0.86      0.79      0.82        70
           1       0.73      0.82      0.77        50

    accuracy                           0.80       120
   macro avg       0.80      0.80      0.80       120
weighted avg       0.81      0.80      0.80       120

